# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [3]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import gkey

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [19]:
part1_csvfile = os.path.join("weather_df.csv")
weather_df = pd.read_csv("weather_df")
weather_df.reset_index(drop=True, inplace=True)
weather_df.sort_values(ascending=False, by=["Humidity(%)"])

,City_ID,City,Country,Cloudiness,Date,Humidity(%),Lat,Lng,Max Temp (F),Wind Speed
151,151,Hambantota,LK,75,2021-02-02,100,6.1241,81.1185,73.40,5.75
12,12,Benjamin Constant,BR,40,2021-02-02,100,-4.3831,-70.0311,75.20,0.81
275,275,Māgadi,IN,40,2021-02-02,100,12.9667,77.2333,62.60,2.30
397,397,Horsham,GB,90,2021-02-02,100,51.0640,-0.3272,45.00,8.05
43,43,Mutoko,ZW,100,2021-02-02,100,-17.3970,32.2268,64.89,3.02
...,...,...,...,...,...,...,...,...,...,...
283,283,Atar,MR,5,2021-02-02,20,20.5169,-13.0499,69.35,7.65
318,318,Goundam,ML,50,2021-02-02,20,16.4145,-3.6708,66.69,10.16
485,485,Cody,US,1,2021-02-02,19,44.5263,-109.0565,44.60,3.89
329,329,Manga,BF,0,2021-02-02,19,11.6636,-1.0731,69.01,8.93


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [32]:
# Configure gmaps
gmaps.configure(api_key = gkey)
# set location & heat mapping variable of humidity
locations_map = weather_df[["Lat", "Lng"]]
weight = weather_df["Humidity(%)"].astype(float)
# assign heat layer params
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations=locations_map, weights=weight, dissipating=False,
                                 point_radius=3, max_intensity=100)
# Add heat layer
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
weather_filter = weather_df[(weather_df["Max Temp (F)"]<=85)&
           (weather_df["Max Temp (F)"]>=70)&
           (weather_df["Humidity(%)"]<55)&
           (weather_df["Wind Speed"]<15)&
           (weather_df["Cloudiness"]<50)]
weather_filter.reset_index(drop=True, inplace=True)
weather_filter

,City_ID,City,Country,Cloudiness,Date,Humidity(%),Lat,Lng,Max Temp (F),Wind Speed
0,13,Busselton,AU,0,2021-02-02,46,-33.6500,115.3333,81.00,1.01
1,27,Hilo,US,20,2021-02-02,54,19.7297,-155.0900,80.60,10.36
2,193,Gouré,NE,35,2021-02-02,12,13.9835,10.2704,70.00,8.01
3,217,Launceston,AU,4,2021-02-02,37,-41.4500,147.1667,71.01,5.75
4,243,Mildura,AU,0,2021-02-02,40,-34.2000,142.1500,72.00,12.66
5,245,Verāval,IN,0,2021-02-02,35,20.9000,70.3667,71.80,9.98
6,292,Esperance,AU,0,2021-02-02,53,-33.8667,121.9000,71.60,14.97
7,300,Diofior,SN,0,2021-02-02,29,14.1833,-16.6667,75.20,9.35
8,382,Qandala,SO,0,2021-02-02,48,11.4720,49.8728,71.31,5.39
9,383,Moïssala,TD,0,2021-02-02,25,8.3404,17.7663,73.99,3.04


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [9]:
hotel_df = weather_filter.copy()
hotel_df["Hotel Name"] = ""
hotel_df.reset_index(drop=True,inplace=True)
hotel_df

,City_ID,City,Country,Cloudiness,Date,Humidity(%),Lat,Lng,Max Temp (F),Wind Speed,Hotel Name
0,13,Busselton,AU,0,2021-02-02,46,-33.6500,115.3333,81.00,1.01,
1,27,Hilo,US,20,2021-02-02,54,19.7297,-155.0900,80.60,10.36,
2,193,Gouré,NE,35,2021-02-02,12,13.9835,10.2704,70.00,8.01,
3,217,Launceston,AU,4,2021-02-02,37,-41.4500,147.1667,71.01,5.75,
4,243,Mildura,AU,0,2021-02-02,40,-34.2000,142.1500,72.00,12.66,
5,245,Verāval,IN,0,2021-02-02,35,20.9000,70.3667,71.80,9.98,
6,292,Esperance,AU,0,2021-02-02,53,-33.8667,121.9000,71.60,14.97,
7,300,Diofior,SN,0,2021-02-02,29,14.1833,-16.6667,75.20,9.35,
8,382,Qandala,SO,0,2021-02-02,48,11.4720,49.8728,71.31,5.39,
9,383,Moïssala,TD,0,2021-02-02,25,8.3404,17.7663,73.99,3.04,


In [11]:
import pprint as pprint
target_type = "lodging"
hotel_names = []

# looping through the lat/lng to find hotels by radius:5000.
for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    city = row["City"]
    params = {
    "location": f"{lat},{lng}",
    "type": target_type,
    "radius": 5000,
    "key": gkey
    }
    # API Calls loop
    try:
        print(f"Retrieving Results for {city}")
        base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
        response = requests.get(base_url, params).json()
    # Obtain hotel information. First result is obtained from API call list.         
        hotel_names.append(response["results"][0]["name"])
    except (KeyError, IndexError):
        # Filter & drop cities without hotel results
        hotel_names.append(np.nan)
        print(f"Missing field....skipping {city}")
    print("-----------------------------------------------")

Retrieving Results for Busselton
-----------------------------------------------
Retrieving Results for Hilo
-----------------------------------------------
Retrieving Results for Gouré
Missing field....skipping Gouré
-----------------------------------------------
Retrieving Results for Launceston
-----------------------------------------------
Retrieving Results for Mildura
-----------------------------------------------
Retrieving Results for Verāval
-----------------------------------------------
Retrieving Results for Esperance
-----------------------------------------------
Retrieving Results for Diofior
-----------------------------------------------
Retrieving Results for Qandala
Missing field....skipping Qandala
-----------------------------------------------
Retrieving Results for Moïssala
Missing field....skipping Moïssala
-----------------------------------------------
Retrieving Results for Ban Nahin
Missing field....skipping Ban Nahin
-------------------------------------

In [12]:
# Hotel list filtered out missing results
hotel_df["Hotel Name"] = hotel_names
hotel_df.dropna(inplace=True)
hotel_df

,City_ID,City,Country,Cloudiness,Date,Humidity(%),Lat,Lng,Max Temp (F),Wind Speed,Hotel Name
0,13,Busselton,AU,0,2021-02-02,46,-33.6500,115.3333,81.00,1.01,Observatory Guest House
1,27,Hilo,US,20,2021-02-02,54,19.7297,-155.0900,80.60,10.36,Hilo Hawaiian Hotel
3,217,Launceston,AU,4,2021-02-02,37,-41.4500,147.1667,71.01,5.75,Hotel Grand Chancellor Launceston
4,243,Mildura,AU,0,2021-02-02,40,-34.2000,142.1500,72.00,12.66,Mercure Hotel Mildura
5,245,Verāval,IN,0,2021-02-02,35,20.9000,70.3667,71.80,9.98,Lords Inn Somnath
6,292,Esperance,AU,0,2021-02-02,53,-33.8667,121.9000,71.60,14.97,Hospitality Esperance
7,300,Diofior,SN,0,2021-02-02,29,14.1833,-16.6667,75.20,9.35,Souimanga Lodge
11,493,Nishihara,JP,40,2021-02-02,53,26.1842,127.7558,73.40,9.22,Minshuku Agaihama


In [33]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [35]:
# Add marker layer ontop of heat map
# Assign the marker layer to a variable
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
# Add the layer to the map
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))